[View in Colaboratory](https://colab.research.google.com/github/lycohen/ds_desafio_I/blob/master/Desafio_III_con_pandas.ipynb)

In [24]:
! pip install pydrive
! pip install sklearn

# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id_15 = '1Oden9as5wDneEm--EvypPFzQ0l1LK2xW'

file_id_16 = '1gCDKeVRyFNTb8DK1_Rxov0j-FniBo5nK'
file_id_17 = '1Kfgnl1E53pXRvw24O2c2Vj9GYIbqE5wZ'
file_id_18= '15tAV6dotJY2nSaxp_x3CCTmpWkZmzS2G'
file_id_19= '16qqIgdVFwgdTIsQcRorMt4X-8fKupqb7'
file_id_20= '15cM8zF7rrVisxtKXlD94EfLuVamkXf_k'
file_id_21= '17HOAScPpOg92-zepqA_p0sJ8iPfU_yru'
file_id_Test =  '1NMI7QW7l63Uq42yd3ytBaNG224auwi23'


downloadedF15  = drive.CreateFile({'id': file_id_15})
downloadedF16  = drive.CreateFile({'id': file_id_16})
downloadedF17  = drive.CreateFile({'id': file_id_17})
downloadedF18  = drive.CreateFile({'id': file_id_18})
downloadedF19  = drive.CreateFile({'id': file_id_19})
downloadedF20  = drive.CreateFile({'id': file_id_20})
downloadedF21  = drive.CreateFile({'id': file_id_21})
downloadedFTest  = drive.CreateFile({'id': file_id_Test})

# allows you to temporarily load your file in the notebook VM

# assume the file is called file.csv and it's located at the root of your drive
downloadedF15.GetContentFile('ctr_15.zip')
downloadedF16.GetContentFile('ctr_16.zip')
downloadedF17.GetContentFile('ctr_17.zip')
downloadedF18.GetContentFile('ctr_18.zip')
downloadedF19.GetContentFile('ctr_19.zip')
downloadedF20.GetContentFile('ctr_20.zip')
downloadedF21.GetContentFile('ctr_21.zip')
downloadedFTest.GetContentFile('ctr_test_labeled.zip')
#print (file)

In [0]:
import pandas as pd
import zipfile
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import FeatureHasher
from scipy.sparse import coo_matrix, vstack
import numpy as np
from sklearn import linear_model as lm
import random


In [0]:
def getDfFeatures(df):
  return df.iloc[:,1:]

def getDfLabels(df):
  return df.iloc[:,0:1].values

In [0]:
def getDf(arch):
  return pd.read_csv(zipfile.ZipFile(arch+'.zip').open(arch+'.csv'))

df = pd.concat([getDf('ctr_15'),getDf('ctr_16')],axis=0)


In [0]:
def fillna(X):
  X['bid_floor_group']= pd.cut(X.auction_bidfloor.values,4,labels=['1q','2q', '3q', '4q'])
  X['auction_age_group'] = pd.cut(X.auction_age.fillna(-1).values,4,labels=['1q','2q', '3q', '4q'])
  X.auction_time=X.auction_time/1000000000
  #X.creative_height = X.creative_height.fillna(-1)        
  #X.creative_width  = X.creative_width.fillna(-1) 
  #X.timezone_offset =X.timezone_offset.fillna(-1)  #for col in X.columns:
  
  for i in X.columns:
    if (X[i].isna().sum()):
      X[i] =X[i].fillna(-1)
  X=X.drop(['auction_bidfloor','auction_age','action_list_1','action_list_2','auction_list_0'],axis=1)
   
  return X

df = fillna(df)

In [0]:
####Con el feature hasher de sklearn
vec = FeatureHasher(n_features=5000,input_type='dict',dtype =np.float64, non_negative=False)
#clf2 = lm.SGDClassifier(loss='log',penalty = 'l2',fit_intercept=True,n_jobs=64,warm_start=False,shuffle=False,verbose=0,n_iter=2) # shuffle=True is useless here
clf2 = lm.SGDClassifier(loss='log',penalty = 'l2',fit_intercept=True,n_jobs=64,warm_start=False,shuffle=False,verbose=0) # shuffle=True is useless here


In [0]:



train, test = train_test_split(df, test_size=0.1)  
Ytrain = train['Label'].values
Ytest = test['Label'].values


In [0]:
Xttest = vec.fit_transform(test.iloc[:,1:].to_dict(orient='records'))


In [32]:
display(Xttest,len(Ytest))

<233194x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 10927601 stored elements in Compressed Sparse Row format>

233194

In [33]:

df = train
nicial=-1
paso=2000
scores=[]

for n in range(1,df.shape[0]-1,paso):
  nicial=nicial+paso
  #print(n-1,nicial)
  X=train.iloc[n-1:nicial,1:]
  Yt=train.iloc[n-1:nicial,:].Label
  dictX = X[X.columns].to_dict(orient='records')
  try:
    Xt = vec.fit_transform(dictX)
   
    clf2.partial_fit(Xt,Yt,classes=np.unique(Yt));
    #print(clf2.score(Xttest,Ytest))
    scores.append(clf2.score(Xttest,Ytest))
  except Exception as e :
    #pass
    print (str(e))#print(dictX)#Y=df.iloc[n-1:nicial,:].Label
  #clf2.partial_fit(Xt, Y, classes=np.unique(Y))
 
  

  #print (clf2.score(Xt, Y))





/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [35]:
scores=pd.DataFrame(scores,columns=['score'])
scores.describe()

,score
count,1050.000000
mean,0.980708
std,0.064090
min,0.121680
25%,0.988571
50%,0.989751
75%,0.989884
max,0.989978


2331937

In [0]:
y_train=getDfFeatures(train)
x_train=getDfLabels(train)

In [0]:
##Listamos las columnas por tipo
#g = df.columns.to_series().groupby(df.dtypes).groups

    

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler

class MyScaler(BaseEstimator, TransformerMixin):
  def __init__(self, cols):
    self.cols = cols

  def fit(self, X, y=None):
    #self.scaler = MinMaxScaler()
    self.scaler = StandardScaler()
    self.scaler.fit(X[self.cols])
    return self

  def transform(self, X):
    return self.scaler.transform(X[self.cols])

In [0]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import HashingVectorizer

class MyVectorizer(BaseEstimator, TransformerMixin):
  
  def __init__(self, cols, hashing=None):
    """
    args:
    cols: a list of column names of the categorical variables
    hashing: 
    If None, then vectorization is a simple one-hot-encoding.
    If an integer, then hashing is the number of features in the output.
        """
    self.cols = cols
    self.hashing = hashing

  def fit(self, X, y=None):

    data = X[self.cols]

    # Choose a vectorizer
    if self.hashing is None:
        self.myvec = HashingVectorizer()
    else:
        #self.myvec = FeatureHasher(n_features = self.hashing)
        self.myvec = FeatureHasher(n_features = self.hashing, non_negative=True)

    self.myvec.fit(X[self.cols].to_dict(orient='records'))
    return self

  def transform(self, X):

    # Vectorize Input
    if self.hashing is None:
        return pd.DataFrame(
            self.myvec.transform(X[self.cols].to_dict(orient='records')),
            columns = self.myvec.feature_names_
        )
    else:
        return pd.DataFrame(
            self.myvec.transform(X[self.cols].to_dict(orient='records')).toarray()
        )

In [0]:
search_params = {
    'preprocess__vectorize__hashing': [20, 40, 80],
    'predict__alpha': [.01, .1, 1, 2, 10]
}

In [0]:
numeric_predictors=['auction_age', 'auction_bidfloor']

In [0]:
categorical_predictors=['action_categorical_0', 'action_categorical_1', 'action_categorical_2',
        'action_categorical_3', 'action_categorical_4', 'action_categorical_5',
        'action_categorical_6', 'action_categorical_7', 'action_list_0',
        'action_list_1', 'action_list_2', 'auction_boolean_0',
        'auction_boolean_1', 'auction_boolean_2', 'auction_categorical_0',
        'auction_categorical_1', 'auction_categorical_10',
        'auction_categorical_11', 'auction_categorical_12',
        'auction_categorical_2', 'auction_categorical_3',
        'auction_categorical_4', 'auction_categorical_5',
        'auction_categorical_6', 'auction_categorical_7',
        'auction_categorical_8', 'auction_categorical_9', 'auction_list_0',
        'creative_categorical_0', 'creative_categorical_1',
        'creative_categorical_10', 'creative_categorical_11',
        'creative_categorical_12', 'creative_categorical_2',
        'creative_categorical_3', 'creative_categorical_4',
        'creative_categorical_5', 'creative_categorical_6',
        'creative_categorical_7', 'creative_categorical_8',
        'creative_categorical_9', 'device_id_type', 'gender','creative_height']

In [0]:
otros = ['auction_time','creative_height','timezone_offset', 'creative_width']

In [128]:
#!pip install scikit-learn==0.20.0
!pip install utils
 

In [0]:
import operator
from functools import reduce
from itertools import product as iproduct
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier


def product(iterable, start=1):
    return reduce(operator.mul, iterable, start)


class PolyDictVectorizer(DictVectorizer):

    def __init__(self, degree=2, sparse=True, num_types=[float]):
        self.degree = degree
        self.num_types = num_types
        super().__init__(sparse=sparse)

    def fit(self, X, y=None):
        ###agregado por mi para transformar el df a diccionario.
        X = X.to_dict(orient='records')
        return super().fit(map(self._encode, X), y)

    def _transform(self, X, fitting):
        return super()._transform(map(self._encode, X), fitting)

    def _encode(self, dic):
        dic = {k if type(v) in self.num_types else f'{k}={v}':
               float(v) if type(v) in self.num_types else 1
               for k, v in dic.items()}
        dic_keys = list(dic.keys())
        for deg in range(2, self.degree + 1):
            for term_keys in iproduct(dic_keys, repeat=deg):
                term_names, term_facts = [], []
                for k, n in Counter(term_keys).items():
                    v = dic[k]
                    if type(v) is int and n > 1:
                        break
                    term_names.append(k if n == 1 else f'{k}^{n}')
                    term_facts.append(v**n)
                else:  # No dummy feature was included more than once
                    dic['*'.join(sorted(term_names))] = product(term_facts)
        return dic



In [0]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import Imputer

pipeline = Pipeline([
    ('preprocess', FeatureUnion([
        #('imp', Imputer(strategy='most_frequent', axis=0)),
        #('scale', MyScaler(cols=numeric_predictors)),
        #('vectorize', MyVectorizer(cols=categorical_predictors, hashing=5))
        ('poly',PolyDictVectorizer())
    ])),
    ('predict', SGDClassifier())
])

In [146]:
x = pipeline.fit(y_train)

AttributeError: ignored

In [111]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline, search_params)
grid_search.fit(x_train,y_train)

TypeError: ignored

AttributeError: ignored

In [51]:
SGDClassifier()
import numpy as np
# SGD mini-batch
# Get num training samples
df_train=df
N_train = len(df_train)
num_passes = 50
output='Label'
batch_size = N_train
model = sklearn.linear_model.SGDClassifier(loss= 'log', alpha = 1, tol = 0.00001, shuffle = False, random_state = 0)
# Set 80% of data to belong to training set; reserve the rest for validation set
train_indices = list(range(int(0.8*N_train)))
valid_indices = list(range(int(0.8*N_train), N_train))

sgd_minibatch_scores = []
for i in range(num_passes):
    np.random.shuffle(train_indices)
    for j in range(int(len(train_indices)/batch_size + 1)):

        if j == int(len(train_indices)/batch_size+1):
            batch_train_indices = train_indices[j*batch_size:]
        else:
            batch_train_indices = train_indices[j*batch_size:(j+1)*batch_size]

        model.partial_fit(df_train.iloc[batch_train_indices], df_train[output].iloc[batch_train_indices], classes = [0,1],)

    sgd_minibatch_scores.append(model.score(df_train.iloc[valid_indices], df_train[output].iloc[valid_indices]))


ValueError: ignored

In [27]:
vec = PolyDictVectorizer()

for i in range(0,len(df),1000):
  limiteSup = i + 1000
  dfChunk = df.iloc[i:limiteSup]
  features =getDfFeatures(dfChunk)
  labels = getDfLabels(dfChunk)
  dictFeat=features.to_dict(orient='records')
   vec.fit(dictFeat)
  

KeyboardInterrupt: ignored

(1239524, 1)

(1239524, 51)

In [0]:
###NO FUNCIONA.


In [13]:
vec

PolyDictVectorizer(degree=2, num_types=[<class 'float'>], sparse=True)

NameError: ignored

In [4]:
dfFull  =  pd.concat([pd.read_csv(zipfile.ZipFile('ctr_15.zip').open('ctr_15.csv')), pd.read_csv(zipfile.ZipFile('ctr_16.zip').open('ctr_16.csv')),pd.read_csv(zipfile.ZipFile('ctr_17.zip').open('ctr_17.csv')),pd.read_csv(zipfile.ZipFile('ctr_18.zip').open('ctr_18.csv')),pd.read_csv(zipfile.ZipFile('ctr_19.zip').open('ctr_19.csv')),pd.read_csv(zipfile.ZipFile('ctr_20.zip').open('ctr_20.csv')), pd.read_csv(zipfile.ZipFile('ctr_21.zip').open('ctr_21.csv'))],axis=0)


UsageError: Line magic function `%memit` not found.


In [21]:
##tamaño del df en GB
import sys
dfFull.memory_usage(index=True).sum()/1000000000 #giga
#float(sys.getsizeof(dfFull)) 


3.604794864

In [0]:
%reset dfFull

In [1]:
dict=dfFull.to_dict()


NameError: ignored

In [9]:
dfFull.sample(10)

,Label,action_categorical_0,action_categorical_1,action_categorical_2,action_categorical_3,action_categorical_4,action_categorical_5,action_categorical_6,action_categorical_7,action_list_0,...,creative_categorical_7,creative_categorical_8,creative_categorical_9,creative_height,creative_width,device_id,device_id_type,gender,has_video,timezone_offset
1009363,0,9915ffee,dc24b79b,d2f34a41,7ce4e1a3,b55cb32e,6bc0e29c,59638795,e2538fca,IAB22-2,...,NaN,b6910b48,65dcab89,50.0,320.0,9146e66e,c1d12c8e,NaN,False,1.0
1208071,0,11b7af3d,0d132da2,63fa8c3e,f51428b0,75289988,6bc0e29c,59638795,31b31f57,IAB22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392457,0,c186959e,11fe6f26,ca8353da,93896d75,113da814,79ceee49,59638795,e2538fca,IAB8-9,...,NaN,b6910b48,65dcab89,50.0,320.0,d0d9118a,c1d12c8e,m,False,1.0
251755,0,c2e4f717,3074db21,b81de479,62c903fc,a789c7fd,6bc0e29c,59638795,e2538fca,IAB20-6,...,NaN,b6910b48,43c867fd,480.0,320.0,15372651,c1d12c8e,NaN,False,1.0
302433,0,c2e4f717,e709bbc0,eafe914f,e7329a92,ea29c22f,6bc0e29c,59638795,e2538fca,IAB20-6,...,NaN,b6910b48,65dcab89,250.0,300.0,5db49d16,c1d12c8e,NaN,False,1.0
1028320,0,604d011f,f71d2f9b,9e4f5826,2c66682b,c3ab0db6,79ceee49,59638795,e2538fca,IAB8-9,...,f1b6690d,b00371d3,NaN,NaN,NaN,cbbec9fe,6324b367,NaN,False,1.0
392597,0,c2e4f717,3074db21,d724f567,62c903fc,2ceafe2f,6bc0e29c,59638795,e2538fca,IAB20-6,...,NaN,b6910b48,65dcab89,50.0,320.0,67f8470d,6324b367,m,False,1.0
929785,0,c2e4f717,e709bbc0,75f72165,e7329a92,99e030f3,6bc0e29c,59638795,e2538fca,IAB20-6,...,NaN,b6910b48,65dcab89,250.0,300.0,32d5f490,6324b367,NaN,False,1.0
1002777,0,11b7af3d,ac0f362d,2f2b9920,cb80abab,2c3ce269,6bc0e29c,59638795,31b31f57,IAB22,...,NaN,b6910b48,43c867fd,480.0,320.0,64aa8b3e,6324b367,NaN,False,1.0
1095034,0,e350c7c7,e220fce9,880f2cbf,e9c266f7,c94e1583,6bc0e29c,59638795,31b31f57,IAB9-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
dfFull.shape

(2331937, 52)